In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
MODEL_PATH = "cnn_model.h5"
LABEL_ENCODER_PATH = "label_encoder.npy"

IMAGE_PATH = "/content/drive/MyDrive/faysond/images_original"
IMG_SIZE = (128, 128)

data = []
genres = os.listdir(IMAGE_PATH)

for genre in genres:
    genre_path = os.path.join(IMAGE_PATH, genre)
    if not os.path.isdir(genre_path):
        continue

    for file in os.listdir(genre_path):
        file_path = os.path.join(genre_path, file)
        try:
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, IMG_SIZE)
            data.append([img, genre])
        except Exception as e:
            print(f"Skipping {file_path} (Error: {e})")

In [ ]:
df = pd.DataFrame(data, columns=["features", "genre"])
df["features"] = df["features"].apply(lambda x: np.array(x))
encoder = LabelEncoder()
df["genre"] = encoder.fit_transform(df["genre"])

X = np.stack(df["features"].values)
y = df["genre"].values

In [ ]:
X = X / 255.0

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f" Images successfully loaded!")
print(f" Training set: {X_train.shape}")
print(f" Validation set: {X_val.shape}")
print(f" Test set: {X_test.shape}")

X_train = X_train.reshape(-1, 128, 128, 1)
X_val = X_val.reshape(-1, 128, 128, 1)
X_test = X_test.reshape(-1, 128, 128, 1)

 Images successfully loaded!
 Training set: (799, 128, 128)
 Validation set: (100, 128, 128)
 Test set: (100, 128, 128)


In [ ]:
cnn_model = Sequential([
    Conv2D(32, kernel_size=(3,3), activation="relu", input_shape=(128, 128, 1)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64, kernel_size=(3,3), activation="relu"),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(128, kernel_size=(3,3), activation="relu"),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.3),
    Dense(len(np.unique(y)), activation="softmax")  
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
cnn_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
cnn_model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.0843 - loss: 2.3181 - val_accuracy: 0.2600 - val_loss: 2.1502
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.2261 - loss: 2.1273 - val_accuracy: 0.2700 - val_loss: 1.9475
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.2927 - loss: 1.9708 - val_accuracy: 0.2500 - val_loss: 1.9126
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.3513 - loss: 1.8291 - val_accuracy: 0.3200 - val_loss: 1.8081
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.3800 - loss: 1.7766 - val_accuracy: 0.3900 - val_loss: 1.7459
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.4267 - loss: 1.6037 - val_accuracy: 0.3900 - val_loss: 1.8139
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.4726 - loss: 1.5026 - val_accuracy: 0.3700 - val_loss: 1.6131
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.5448 - loss: 1.3341 - val_accuracy: 0.4100 - val_loss:

In [ ]:
cnn_model.fit(X_train, y_train, epochs=17, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/18
25/25 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.9691 - loss: 0.1082 - val_accuracy: 0.5600 - val_loss: 2.2249
Epoch 2/18
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.9614 - loss: 0.1165 - val_accuracy: 0.5500 - val_loss: 2.1299
Epoch 3/18
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.9694 - loss: 0.0916 - val_accuracy: 0.5400 - val_loss: 2.1171
Epoch 4/18
25/25 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.9529 - loss: 0.1164 - val_accuracy: 0.5100 - val_loss: 2.0401
Epoch 5/18
25/25 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.9718 - loss: 0.0965 - val_accuracy: 0.5300 - val_loss: 2.4966
Epoch 6/18
25/25 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.9708 - loss: 0.0832 - val_accuracy: 0.5700 - val_loss: 1.9900
Epoch 7/18
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.9536 - loss: 0.1470 - val_accuracy: 0.5900 - val_loss: 2.2999
Epoch 8/18
25/25 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.9734 - loss: 0.0868 - val_accuracy: 0.5800 - val_loss:

In [ ]:
loss, cnn_accuracy = cnn_model.evaluate(X_test, y_test)
print(f"\n CNN Model Accuracy: {cnn_accuracy * 100:.2f}%")

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step - accuracy: 0.6214 - loss: 1.3634

 CNN Model Accuracy: 59.00%


In [ ]:
cnn_model.save(MODEL_PATH)
print(f" Model saved as {MODEL_PATH}")

 Model saved as cnn_model.h5


In [ ]:
np.save(LABEL_ENCODER_PATH, encoder.classes_)
print(f" Label encoder saved as {LABEL_ENCODER_PATH}")

 Label encoder saved as label_encoder.npy
